In [1]:
%load_ext autoreload
%autoreload 2

import torch

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from ml_rnn_names.model import CharLSTM
from ml_rnn_names.processing import n_letters
from ml_rnn_names.training import train
from ml_rnn_names.utils import torch_device_setup
from ml_rnn_names.data import load_data, collate_names

In [ ]:
# Perform Random Search to find suitable hyperparameters
device = torch_device_setup()
train_set, _, test_set, labels_uniq = load_data(device)

n_trials = 5  # Number of random combinations to try

param_ranges = {
    "hidden_size": [64, 128, 256, 512],  # discrete choices
    "learning_rate": (0.001, 0.5),  # (min, max) for uniform sampling
    "n_batch_size": [32, 64, 128],  # discrete choices
    "num_layers": (2, 5),  # number of lstm layers
    "n_epoch": (5, 20),  # will be converted to int
}  # I removed "learning_rate" to try multiple below

results = []
plt.figure(figsize=(12, 6))

for trial in range(n_trials):
    # Randomly sample hyperparameters
    hidden_size = int(np.random.choice(param_ranges["hidden_size"]))
    num_layers = int(np.random.uniform(*param_ranges["num_layers"]))
    learning_rate = np.random.uniform(*param_ranges["learning_rate"])
    n_batch_size = int(np.random.choice(param_ranges["n_batch_size"]))
    n_epoch = int(np.random.uniform(*param_ranges["n_epoch"]))

    model_parameters = {
        "input_size": n_letters,
        "output_size": len(labels_uniq),
        "num_layers": num_layers,
        "hidden_size": hidden_size,
    }
    hyper_parameters = {
        "n_epoch": n_epoch,
        "n_batch_size": n_batch_size,
        "learning_rate": learning_rate,
    }

    print(
        f"Trial {trial + 1}/{n_trials}: LR={learning_rate:.4f}, "
        f"Hidden={hidden_size}, Batch={n_batch_size}, Epochs={n_epoch}"
    )

    lstm = CharLSTM(**model_parameters)
    result = train(
        model=lstm, training_data=train_set, **hyper_parameters, device=device
    )
    all_losses = result["train_losses"]

    # Store results
    results.append(
        {
            "learning_rate": learning_rate,
            "hidden_size": hidden_size,
            "n_batch_size": n_batch_size,
            "n_epoch": n_epoch,
            "final_loss": all_losses[-1],
            "all_losses": all_losses,
        }
    )

    plt.plot(all_losses, alpha=0.5, label=f"LR={learning_rate:.3f}, H={hidden_size}")

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Training Loss for Random Search")
plt.tight_layout()
plt.show()

# Find best hyperparameters
best_trial = min(results, key=lambda x: x["final_loss"])
print("\nBest hyperparameters:")
for key, value in best_trial.items():
    if key != "all_losses":
        print(f"  {key}: {value}")

In [ ]:
device = torch_device_setup()
train_set, _, test_set, labels_uniq = load_data(device)

model_parameters = {
    "input_size": n_letters,
    "output_size": len(labels_uniq),
    "hidden_size": 128,
    "num_layers": 2,
}
hyper_parameters = {
    "n_epoch": 16,
    "n_batch_size": 32,
    "learning_rate": 0.215,
}


lstm = CharLSTM(**model_parameters)
result = train(
    model=lstm, training_data=train_set, **hyper_parameters, device=device
)
all_losses = result["train_losses"]

plt.plot(all_losses, alpha=0.5, label=f"LR={hyper_parameters['learning_rate']:.3f}, H={model_parameters['hidden_size']}")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.tight_layout()
plt.show()

In [ ]:
from torch.utils.data import DataLoader


def evaluate(model, testing_data, classes):
    confusion = torch.zeros(len(classes), len(classes))

    dataloader = DataLoader(
        testing_data,
        batch_size=64,
        shuffle=False,
        collate_fn=collate_names,
    )

    model.eval()
    with torch.no_grad():
        for labels, padded_texts, lengths in dataloader:
            outputs = model(padded_texts, lengths)
            _, predictions = outputs.topk(1)
            predictions = predictions.squeeze(1)

            for label, pred in zip(labels, predictions):
                confusion[label][pred] += 1

    # Normalize by dividing every row by its sum
    for i in range(len(classes)):
        denom = confusion[i].sum()
        if denom > 0:
            confusion[i] = confusion[i] / denom

    # Set up plot
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(confusion.cpu().numpy())
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticks(np.arange(len(classes)), labels=classes, rotation=90)
    ax.set_yticks(np.arange(len(classes)), labels=classes)

    # Force label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


evaluate(lstm, test_set, classes=labels_uniq)